In [1]:
%%writefile models.py
from typing import List

class Movie:
    def __init__(self, movie_list: List[str]):
        self.movie_list: List[str] = movie_list

    def show_list(self) -> None:
        for i, title in enumerate(self.movie_list):
            print(f"영화 {i} : {title}")

    def get_title(self, index: int) -> str:
        return self.movie_list[index]

class Ticket:
    def __init__(self, movie_name: str, num_people: int, age_list: List[int], seat: str):
        self.movie_name: str = movie_name
        self.num_people: int = num_people
        self.age_list: List[int] = age_list
        self.seat: str = seat
        self.adult: int = 0
        self.child: int = 0
        self.price: int = self.calculate()

    def calculate(self) -> int:
        for age in self.age_list:
            if age < 20:
                self.child += 1
            else:
                self.adult += 1
        return self.adult * 10000 + self.child * 5000

Writing models.py


In [2]:
%%writefile utils.py
from typing import Generator, Any

def get_valid_int(prompt: str) -> int:
    while True:
        try:
            value = int(input(prompt))
            return value
        except ValueError:
            print("잘못된 입력입니다. 숫자를 입력해주세요.")

def log_generator(ticket: Any) -> Generator[str, None, None]:
    yield "----------------------------------"
    yield "티켓 생성을 시작합니다..."
    yield f"영화: {ticket.movie_name}"
    yield f"인원: {ticket.num_people}명 (어른 {ticket.adult}, 아이 {ticket.child})"
    yield f"좌석: {ticket.seat}"
    yield f"총액: {ticket.price}원"
    yield "결제 중..."
    yield "----------------------------------"

Writing utils.py


In [13]:
import asyncio
from typing import List
from models import Movie, Ticket
from utils import get_valid_int, log_generator

async def payment():
  print("결제 중")
  await asyncio.sleep(2)
  print("결제 완료")

async def main() -> None:
    movie_list: List[str] = ["주토피아","아바타","도라에몽","오버워치","동물의숲","going home","내가 제일 잘 나가","개발자란","d-2","내 휴일이 어디갔지"]
    movie_mgr = Movie(movie_list)

    movie_mgr.show_list()
    while True:
        try:
            movie_choice = get_valid_int("영화를 선택하세요 : ")
            selected_title = movie_mgr.get_title(movie_choice)
            break
        except IndexError:
            print("없는 영화 번호입니다. 다시 선택해주세요.")

    print(f"{selected_title} 선택했습니다.")
    print("----------------------------------")

    num_people = get_valid_int("인원 수 입력 : ")
    print(f"{num_people}명 선택했습니다.")
    print("----------------------------------")

    while True:
        try:
            input_age = input("나이를 입력하세요 (공백 구분): ").split()
            if len(input_age) != num_people:
                print(f"인원 수({num_people}명)에 맞춰서 나이를 입력해주세요.")
                continue
            age_list: List[int] = [int(age) for age in input_age]
            break
        except ValueError:
            print("나이는 숫자로 입력해야 합니다.")

    seat: str = input("좌석 선택(A~Z) : ")
    print(f"{seat}좌석 선택했습니다.")

    ticket = Ticket(selected_title, num_people, age_list, seat)

    logger = log_generator(ticket)
    for log in logger:
        print(log)

    while True:
        input_money = get_valid_int("돈을 넣어주세요 : ")

        if input_money < ticket.price:
            print("돈이 부족합니다.")
        else:
            asyncio.run(payment())
            change = input_money - ticket.price
            print(f"거스름돈 : {change}")
            print("----------------------------------")
            print("즐거운 관람되세요.")
            break
await main()


영화 0 : 주토피아
영화 1 : 아바타
영화 2 : 도라에몽
영화 3 : 오버워치
영화 4 : 동물의숲
영화 5 : going home
영화 6 : 내가 제일 잘 나가
영화 7 : 개발자란
영화 8 : d-2
영화 9 : 내 휴일이 어디갔지
영화를 선택하세요 : 4
동물의숲 선택했습니다.
----------------------------------
인원 수 입력 : 3
3명 선택했습니다.
----------------------------------
나이를 입력하세요 (공백 구분): 12 25 3
좌석 선택(A~Z) : A
A좌석 선택했습니다.
----------------------------------
티켓 생성을 시작합니다...
영화: 동물의숲
인원: 3명 (어른 1, 아이 2)
좌석: A
총액: 20000원
결제 중...
----------------------------------
돈을 넣어주세요 : 5656
돈이 부족합니다.
돈을 넣어주세요 : 80000


RuntimeError: asyncio.run() cannot be called from a running event loop